<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки-+-Общие-выводы" data-toc-modified-id="Чек-лист-проверки-+-Общие-выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки + Общие выводы</a></span></li></ul></div>

# Защита персональных данных клиентов

Нужно защитить данные клиентов страховой компании. Необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Необходимо обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

Импорт необходимых библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df.head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


В таблице данных компании соблюден почти идеальный баланс между мужчинами и женщинами. Средний возраст застрахованного составляет около 31 года, при этом медиана составляет 30. Границы возраста застрахованных приводят к выводу, что компания не занимается страхованием лиц, чей возраст составляет менее 18 и старше 65 лет. Также интересно взглянуть на уровень заработной платы застрахованных - медиана составляет чуть более 40 000 рублей. В целом, в компании скорее представлен средний экономический класс, чем слишком бедные или слишком богатые граждане.

In [6]:
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [7]:
df.duplicated().sum()

153

Типы данных представлены верно, пропусков не обнаружено. Обнаружено 153 дубликата

In [8]:
df.sort_values(by='Зарплата')[df.duplicated(keep=False)]

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
2955,1,32.0,21600.0,0,0
2988,1,32.0,21600.0,0,0
361,0,50.0,24700.0,1,2
2869,0,50.0,24700.0,1,2
333,0,32.0,25600.0,1,0
...,...,...,...,...,...
1542,0,40.0,54000.0,2,0
1042,0,33.0,59300.0,2,0
2231,0,33.0,59300.0,2,0
1418,1,28.0,59900.0,0,0


Лица в датафрейме повторяются (но могут быть и просто очень похожие люди). Поскольку данные одинаковые, то их можно удалить, не вредя выборке, так как уникальных данных они в себе не содержат.

In [9]:
df = df.drop_duplicates()

In [10]:
df.sort_values(by='Зарплата')[df.duplicated(keep=False)]

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты


In [11]:
df = df.rename(columns={'Пол': 'пол', 'Возраст': 'возраст', 'Зарплата': 'зарплата', 'Члены семьи': 'члены_семьи', 'Страховые выплаты': 'страховые_выплаты'})
df.head()

,пол,возраст,зарплата,члены_семьи,страховые_выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [12]:
df['возраст'].corr(df['страховые_выплаты'])

0.6549641033747963

Между возрастом и количеством страховых выплат существует высокая положительная корреляция. То есть чем больше возраст застрахованного лица, тем больше он имеет выплат.

**Выводы**

1. Типы данных представлены верно, пропусков не обнаружено. Обнаружено 153 дубликата
2. Лица в датафрейме повторяются (но могут быть и просто очень похожие люди). Поскольку данные одинаковые, то их можно удалить, не вредя выборке, так как уникальных данных они в себе не содержат.
3. Названия столбцов стандартизированы.
4. Между возрастом и количеством страховых выплат существует высокая положительная корреляция. То есть чем больше возраст застрахованного лица, тем больше он имеет выплат.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?**

In [13]:
df.head()

,пол,возраст,зарплата,члены_семьи,страховые_выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [14]:
X = df.drop('страховые_выплаты', 1)
y = df['страховые_выплаты']

In [15]:
# добавим 1 вектор, что соответствует w0
X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
X.shape

(4847, 5)

Создададим различные варианты матриц для дальнейшего домножения на X.
Также посмотрим детерминант матриц , что будет подтверждать обратима или необратима матрица (если det = 0, то необратима)

X - исходная матрица;
E - единичная матрица;
R - рандомная(обратимая) матрица. 


In [16]:
# функция проверки матрицы на обратимость
def error_inv(X):
    try: 
        np.linalg.inv(X)
        print("Матрица обратима, детерминант = ",np.linalg.det(X))
    except:
        print("Матрица необратима, или не удовлетворяет требованиям к обратимой матрице")

In [17]:
error_inv(X)

Матрица необратима, или не удовлетворяет требованиям к обратимой матрице


In [18]:
E = np.eye(5)
E

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [19]:
error_inv(E)

Матрица обратима, детерминант =  1.0


In [20]:
np.random.seed(123)
R = np.random.normal(size=(5, 5))
R

array([[-1.0856306 ,  0.99734545,  0.2829785 , -1.50629471, -0.57860025],
       [ 1.65143654, -2.42667924, -0.42891263,  1.26593626, -0.8667404 ],
       [-0.67888615, -0.09470897,  1.49138963, -0.638902  , -0.44398196],
       [-0.43435128,  2.20593008,  2.18678609,  1.0040539 ,  0.3861864 ],
       [ 0.73736858,  1.49073203, -0.93583387,  1.17582904, -1.25388067]])

In [21]:
error_inv(R)

Матрица обратима, детерминант =  3.841960128348786


In [22]:
X_R = X.dot(R)
X_R.shape

(4847, 5)

In [23]:
T = X.T.dot(X)
T.shape

(5, 5)

In [24]:
X_T = X.dot(T)
X_T.shape

(4847, 5)

In [25]:
Z = np.zeros((5,5))
Z

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [26]:
error_inv(Z)

Матрица необратима, или не удовлетворяет требованиям к обратимой матрице


In [27]:
error_inv(X.dot(Z))

Матрица необратима, или не удовлетворяет требованиям к обратимой матрице


Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

$$
y = X * (X^T X)^{-1} X^T y 
$$

Упрощение:
$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w', где P - случайная обратимая матрица.
$$
$$
w = (X^T X)^{-1} X^T y
$$
$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$

$$
w' = P^{-1}(X^TX)^{-1} (P^T)^{-1} P^T X^T y
$$

По свойству единичной матрицы:
$$
(P^T)^{-1} P^T = E = 1
$$

$$
w' = P^{-1}(X^T X)^{-1}X^T y
y* = XPP^{-1}(X^T X)^{-1}X^T y, где PP^{-1} = E
$$

При упрощении уравнения X.dot(P) получаем:
$$
y* = X * (X^T X)^{-1} X^T y 
$$



*Таким образом формула y полность совпадает с формулой y*

**Из этого можно сделать вывод, что независимо от того, какую метрику будем использовать MSE,MAE или R2 значения метрик не изменятся!
То есть качесвто линейной регрессии не изменится**

**Ответ:** Качесвто линейной регрессии не изменится

**Примеры полученных матриц**

Произведение исходной матрицы на обратимую матрицу

In [28]:
def check(X, y, name):
    model = LinearRegression()
    model.fit(X, y)
    predictions = model.predict(X)
    print("Метрика MSE для матрицы",name,"равна",mean_squared_error(y,predictions))
    return mean_squared_error(y,predictions)

In [29]:
check(X, y, 'X')

Метрика MSE для матрицы X равна 0.125272638266816


0.125272638266816

In [30]:
check(X_R, y, 'X_R')

Метрика MSE для матрицы X_R равна 0.12527256772236653


0.12527256772236653

Произведение исходной матрицы на необратимую матрицу

In [31]:
check(X.dot(Z), y, name="Z")

Метрика MSE для матрицы Z равна 0.2198540875868632


0.2198540875868632

**Выводы**

1. метрика mse показывает худшие результаты с необратимыми матрицами
2. При умножении матрицы на обратимую матрицу независимо от того, какую метрику будем использовать MSE,MAE или R2 значения метрик не изменятся, то есть качесвто линейной регрессии не изменится

## Алгоритм преобразования

**Алгоритм**



Алгоритм решение заключается в умножения матрицы признаков на случайную обратимую матрицу. Пример такой матрицы уже есть - это X_R, который равен X @ R

In [32]:
error_inv(R)

Матрица обратима, детерминант =  3.841960128348786


In [33]:
error_inv(X_R)

Матрица необратима, или не удовлетворяет требованиям к обратимой матрице


In [34]:
def algorithm(X):
    np.random.seed = 12345
    new_x = X.dot(np.random.normal(size=(5,5)))
    return new_x

In [35]:
X_N = algorithm(X)
X_N.shape

(4847, 5)

**Обоснование**

Данный метод обосновывается тем, что при умножение матрицы признаков на обратимую матрицу качество линейной регрессии не меняется.

Аргументация на практике

In [36]:
check(X_N, y, name='X_N')

Метрика MSE для матрицы X_N равна 0.12527258054786358


0.12527258054786358

## Проверка алгоритма

In [37]:
def full_algorithm(X, y, name):
    model = LinearRegression()
    model.fit(X, y)
    predictions = model.predict(X)
    print('Метрика качества модели R2 для матрицы ' + name + ':', r2_score(y, predictions))


In [38]:
full_algorithm(X, y, 'X')

Метрика качества модели R2 для матрицы X: 0.4302010044852068


In [39]:
full_algorithm(X_R, y, 'X_R')

Метрика качества модели R2 для матрицы X_R: 0.4302013253546083


In [40]:
full_algorithm(X_N, y, 'X_N')

Метрика качества модели R2 для матрицы X_N: 0.4302012670182035


**Вывод**

Метрика R2 для выжеприведённых матриц расходится в рамках статистической погрешности, что говорит о том, что модель линейной регрессии работает одинаково хорошо, как с матрицой с исходными признакми, так и с этой же матрицой, помноженной на случайную обратимую матрицу. 
Алгоритм для защиты персональных данных можно внедрять.

## Чек-лист проверки + Общие выводы

1. Типы данных представлены верно, пропусков не обнаружено. Обнаружено 153 дубликата
2. Лица в датафрейме повторяются (но могут быть и просто очень похожие люди). Поскольку данные одинаковые, то их можно удалить, не вредя выборке, так как уникальных данных они в себе не содержат.
3. Названия столбцов стандартизирвоаны
4. метрика mse показывает худшие результаты с необратимыми матрицами
5. При умножении матрицы на обратимую матрицу независимо от того, какую метрику будем использовать MSE,MAE или R2 значения метрик не изменятся, то есть качесвто линейной регрессии не изменится
6. Алгоритм решение заключается в умножения матрицы признаков на случайную обратимую матрицу. Пример такой матрицы уже есть - это X_R, который равен X @ R
7. Данный метод обосновывается тем, что при умножение матрицы признаков на обратимую матрицу качество линейной регрессии не меняется.
8. Метрика R2 для выжеприведённых матриц расходится в рамках статистической погрешности, что говорит о том, что модель линейной регрессии работает одинаково хорошо, как с матрицой с исходными признакми, так и с этой же матрицой, помноженной на случайную обратимую матрицу. 
9. Алгоритм для защиты персональных данных можно внедрять.



